# Graph Scheduling Operations

Examples of scheduling directives used in graph (and other) processing

First, include some libraries

In [ ]:
# Library imports

#
# Import tensor class
#
from fibertree import Payload, Fiber, Tensor, TensorImage

#
# Import display classes/utilities
#
from IPython.display import display # to display images

def displayTensor(t):
    display(TensorImage(t).im)

#
# Helper for data directory
#
import os

data_dir = "../data"

def datafileName(filename):
    return os.path.join(data_dir, filename)



## Graph Inputs

In [ ]:
#
# Function to create graph inputs
#

def create_inputs(display=True):
    
    # Adjacency matrix - Ranks "S" (source) and "D" (destination)

    a = Tensor.fromUncompressed([ "S", "D"],
                                [ [ 0, 1, 1, 0, 0, 0, 0, 1 ],
                                  [ 0, 0, 1, 1, 0, 0, 1, 0 ],
                                  [ 0, 0, 0, 1, 1, 0, 0, 0 ],
                                  [ 0, 0, 0, 0, 1, 1, 0, 0 ],
                                  [ 1, 0, 0, 0, 0, 1, 0, 0 ],
                                  [ 0, 1, 0, 0, 0, 1, 0, 0 ],
                                  [ 0, 0, 0, 1, 0, 1, 0, 0 ],
                                  [ 1, 1, 0, 0, 0, 0, 0, 0 ] ])


    print("Adjacency Matrix")
    displayTensor(a)

    return (a)



# Uniform coordinate space tiling of two ranks (edge blocking)


In [ ]:
# Create inputs

a = create_inputs()

a_s = a.getRoot()



In [ ]:
# Split top rank(source rows) uniformly in coordinate space creating new rank (source tiles)

a_s = a_s.splitUniform(4)
displayTensor(a_s)

In [ ]:
# Split thrid rank (destination columns) in coordinate space creating new rank (destination tiles)

a_s.splitUniformBelow(4, depth=1)
displayTensor(a_s)

In [ ]:
# Swap second rank (source rows) with third rank (destination tiles)
# Result is:
#   - Source tiles
#   - Destination tiles
#   - Source rows
#   - Destination columns

a_s.swapRanksBelow()
displayTensor(a_s)

## Testing area

For running alternative algorithms